In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NE_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,3.105501e+14
1,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,3.108197e+14
2,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,3.105501e+14
3,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,3.105501e+14
4,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,3.105501e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,3.105501e+14
1,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,3.108197e+14
2,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,3.105501e+14
3,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,3.105501e+14
4,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,3.105501e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
572,2238497109,NE,"1125 Industrial St, WEST POINT, NE",34.038191,-118.241723,NaN
610,4381267003,NE,"1155 1st Street, ROGERS, NE",36.322000,-94.116832,NaN
949,4569447000,NE,"310 Main Street, BRIDGEPORT, NE",41.178779,-73.180859,NaN
956,2333657107,NE,"601 N MAIN ST, MADISON, NE",41.842004,-71.407771,NaN
1402,4380927009,NE,"825 MAIN STREET, LYONS, NE",39.410294,-84.578123,NaN
1492,4579927006,NE,"209 East 6th street, KEARNEY, NE",39.367808,-94.348286,NaN
1502,9398168504,NE,"17654 BAY WOOD DR, OMAHA, NE",41.221162,-96.191857,NaN
1624,6173768404,NE,"302 E US Highway 20, Gordon, NE",43.566264,-119.080143,NaN
1627,4464907000,NE,"306 Main St, LOUISVILLE, NE",38.256140,-85.755204,NaN
1672,5339897103,NE,"1133 N 3RD ST, TECUMSEH, NE",33.558127,-112.069837,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('31')] #NE
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/20 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

20


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12119/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
572,2238497109,NE,"1125 Industrial St, WEST POINT, NE",34.038191,-118.241723,[]
610,4381267003,NE,"1155 1st Street, ROGERS, NE",36.322000,-94.116832,[]
949,4569447000,NE,"310 Main Street, BRIDGEPORT, NE",41.178779,-73.180859,[]
956,2333657107,NE,"601 N MAIN ST, MADISON, NE",41.842004,-71.407771,[]
1402,4380927009,NE,"825 MAIN STREET, LYONS, NE",39.410294,-84.578123,[]
1492,4579927006,NE,"209 East 6th street, KEARNEY, NE",39.367808,-94.348286,[]
1502,9398168504,NE,"17654 BAY WOOD DR, OMAHA, NE",41.221162,-96.191857,[310550074523012]
1624,6173768404,NE,"302 E US Highway 20, Gordon, NE",43.566264,-119.080143,[]
1627,4464907000,NE,"306 Main St, LOUISVILLE, NE",38.256140,-85.755204,[]
1672,5339897103,NE,"1133 N 3RD ST, TECUMSEH, NE",33.558127,-112.069837,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_12119/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
572,2238497109,NE,"1125 Industrial St, WEST POINT, NE",34.038191,-118.241723,NaN
610,4381267003,NE,"1155 1st Street, ROGERS, NE",36.322000,-94.116832,NaN
949,4569447000,NE,"310 Main Street, BRIDGEPORT, NE",41.178779,-73.180859,NaN
956,2333657107,NE,"601 N MAIN ST, MADISON, NE",41.842004,-71.407771,NaN
1402,4380927009,NE,"825 MAIN STREET, LYONS, NE",39.410294,-84.578123,NaN
1492,4579927006,NE,"209 East 6th street, KEARNEY, NE",39.367808,-94.348286,NaN
1502,9398168504,NE,"17654 BAY WOOD DR, OMAHA, NE",41.221162,-96.191857,310550074523012
1624,6173768404,NE,"302 E US Highway 20, Gordon, NE",43.566264,-119.080143,NaN
1627,4464907000,NE,"306 Main St, LOUISVILLE, NE",38.256140,-85.755204,NaN
1672,5339897103,NE,"1133 N 3RD ST, TECUMSEH, NE",33.558127,-112.069837,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
572,2238497109,NE,"1125 Industrial St, WEST POINT, NE",34.038191,-118.241723,NaN
610,4381267003,NE,"1155 1st Street, ROGERS, NE",36.322000,-94.116832,NaN
949,4569447000,NE,"310 Main Street, BRIDGEPORT, NE",41.178779,-73.180859,NaN
956,2333657107,NE,"601 N MAIN ST, MADISON, NE",41.842004,-71.407771,NaN
1402,4380927009,NE,"825 MAIN STREET, LYONS, NE",39.410294,-84.578123,NaN
1492,4579927006,NE,"209 East 6th street, KEARNEY, NE",39.367808,-94.348286,NaN
1624,6173768404,NE,"302 E US Highway 20, Gordon, NE",43.566264,-119.080143,NaN
1627,4464907000,NE,"306 Main St, LOUISVILLE, NE",38.256140,-85.755204,NaN
1672,5339897103,NE,"1133 N 3RD ST, TECUMSEH, NE",33.558127,-112.069837,NaN
2383,6008407009,NE,"211 E MAIN ST, PLYMOUTH, NE",35.866717,-76.747979,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
572,2238497109,NaN
610,4381267003,NaN
949,4569447000,NaN
956,2333657107,NaN
1402,4380927009,NaN
1492,4579927006,NaN
1502,9398168504,310550074523012
1624,6173768404,NaN
1627,4464907000,NaN
1672,5339897103,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/NE_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,547459,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,3.105501e+14
1,547460,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,3.108197e+14
2,547461,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,3.105501e+14
3,547462,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,3.105501e+14
4,547463,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,3.105501e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,547459,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,3.105501e+14,NaN
1,547460,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,3.108197e+14,NaN
2,547461,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,3.105501e+14,NaN
3,547462,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,3.105501e+14,NaN
4,547463,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,3.105501e+14,NaN
...,...,...,...,...,...,...,...,...
5799,553258,6609317001,NE,"17525 Arbor Street, OMAHA, NE",41.236225,-96.189536,3.105501e+14,NaN
5800,553259,7946408605,NE,"1045 N 115th St Ste 150, Omaha, NE",41.269145,-96.093016,3.105501e+14,NaN
5801,553260,8534688606,NE,"2110 S 67th St Ste 106, Omaha, NE",41.239763,-96.015807,3.105501e+14,NaN
5802,553261,9113417203,NE,"2605 South 84th St., OMAHA, NE",41.234633,-96.042815,3.105501e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,547459,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,3.105501e+14,NaN,310550074395002.0
1,547460,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,3.108197e+14,NaN,310819693001003.0
2,547461,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,3.105501e+14,NaN,310550074541018.0
3,547462,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,3.105501e+14,NaN,310550074444002.0
4,547463,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,3.105501e+14,NaN,310550074561011.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,4036167203,NE,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,310550074395002.0
1,6046987205,NE,"2225 Q ST, AURORA, NE",40.875357,-97.988962,310819693001003.0
2,7244777109,NE,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,310550074541018.0
3,7268117104,NE,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,310550074444002.0
4,7983549008,NE,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,310550074561011.0
...,...,...,...,...,...,...
5799,6609317001,NE,"17525 Arbor Street, OMAHA, NE",41.236225,-96.189536,310550074492026.0
5800,7946408605,NE,"1045 N 115th St Ste 150, Omaha, NE",41.269145,-96.093016,310550074444004.0
5801,8534688606,NE,"2110 S 67th St Ste 106, Omaha, NE",41.239763,-96.015807,310550070012014.0
5802,9113417203,NE,"2605 South 84th St., OMAHA, NE",41.234633,-96.042815,310550068054016.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
572,2238497109,NE,"1125 Industrial St, WEST POINT, NE",34.038191,-118.241723,NaN
610,4381267003,NE,"1155 1st Street, ROGERS, NE",36.322000,-94.116832,NaN
949,4569447000,NE,"310 Main Street, BRIDGEPORT, NE",41.178779,-73.180859,NaN
956,2333657107,NE,"601 N MAIN ST, MADISON, NE",41.842004,-71.407771,NaN
1402,4380927009,NE,"825 MAIN STREET, LYONS, NE",39.410294,-84.578123,NaN
1492,4579927006,NE,"209 East 6th street, KEARNEY, NE",39.367808,-94.348286,NaN
1624,6173768404,NE,"302 E US Highway 20, Gordon, NE",43.566264,-119.080143,NaN
1627,4464907000,NE,"306 Main St, LOUISVILLE, NE",38.256140,-85.755204,NaN
1672,5339897103,NE,"1133 N 3RD ST, TECUMSEH, NE",33.558127,-112.069837,NaN
2383,6008407009,NE,"211 E MAIN ST, PLYMOUTH, NE",35.866717,-76.747979,NaN


Convert to File

In [19]:
fips_NE = fips_merge_drop

In [20]:
fips_NE.to_csv("../../../data/state_data/geo/geo_fips/23/NE_fips_scraped.csv") 